# Fedbiomed Image Classifier with Differential Privacy with CIFAR10

In this tutorial we will show how an Image classifier with Opacus (https://opacus.ai/) differential privacy engine can be trained with Fed-BioMed. We refer to the Opacus tutorial available here:
https://opacus.ai/tutorials/building_image_classifier

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from fedbiomed.researcher.requests import Requests
req = Requests()
req.list(verbose=True)

2022-01-19 18:44:51,891 fedbiomed INFO - Component environment:
2022-01-19 18:44:51,892 fedbiomed INFO - - type = ComponentType.RESEARCHER
2022-01-19 18:44:52,933 fedbiomed INFO - Messaging researcher_24bd8fc8-05ae-4d8a-914e-7eb6b2591345 successfully connected to the message broker, object = <fedbiomed.common.messaging.Messaging object at 0x10fb9f9d0>
2022-01-19 18:44:52,979 fedbiomed INFO - Listing available datasets in all nodes... 
2022-01-19 18:44:53,016 fedbiomed INFO - log from: node_8c851678-c66f-41b1-a309-80263ab6cab1 / DEBUG - Message received: {'researcher_id': 'researcher_24bd8fc8-05ae-4d8a-914e-7eb6b2591345', 'command': 'list'}
2022-01-19 18:44:53,092 fedbiomed INFO - log from: node_558e6d2e-b8c0-4f8f-bf9a-ad27cc58f787 / DEBUG - Message received: {'researcher_id': 'researcher_24bd8fc8-05ae-4d8a-914e-7eb6b2591345', 'command': 'list'}
2022-01-19 18:45:03,104 fedbiomed INFO - 
 Node: node_8c851678-c66f-41b1-a309-80263ab6cab1 | Number of Datasets: 1 
+---------+-------------+--

{'node_8c851678-c66f-41b1-a309-80263ab6cab1': [{'name': 'CIFAR10',
   'data_type': 'default',
   'tags': ['#CIFAR10', '#dataset'],
   'description': '50.0 percent of CIFAR10 database',
   'shape': [25000, 3, 32, 32]}],
 'node_558e6d2e-b8c0-4f8f-bf9a-ad27cc58f787': [{'name': 'CIFAR10',
   'data_type': 'default',
   'tags': ['#CIFAR10', '#dataset'],
   'description': '50.0 percent of CIFAR10 database',
   'shape': [25000, 3, 32, 32]}]}

## Start the network
Before running this notebook, start the network with `./scripts/fedbiomed_run network`

## Setting the node up
It is necessary to previously configure a node:
1. `./scripts/fedbiomed_run node add`
  * Select option 2 (default), and write CIFAR10 to add CIFAR to the node through `torchvision.datasets.CIFAR10`
  * Confirm default tags by hitting "y" and ENTER
  * Pick the folder where CIFAR is downloaded
  * Data must have been added (if you get a warning saying that data must be unique is because it's been already added)
  
2. Check that your data has been added by executing `./scripts/fedbiomed_run node list`
3. Run the node using `./scripts/fedbiomed_run node run`. Wait until you get `Starting task manager`. it means you are online.

## Create an experiment to train a model on the data found

Declare a TorchTrainingPlan Net class to send for training on the node

In [3]:
from fedbiomed.researcher.environ import environ
import tempfile
tmp_dir_model = tempfile.TemporaryDirectory(dir=environ['TMP_DIR']+'/')
model_file = tmp_dir_model.name + '/Cifar_opacus.py'

In the cell below, we are going to define the model using opacus for differential privacy. For this example, we are going to use the `ModuleValidator` function to validate and/or correct models to be compatible with the `opacus` engine, and the function `make_private_with_epsilon` from `opacus.privacy_engine`. 

To train a model with `make_private_with_epsilon` from Opacus library, there are three privacy-specific hyper-parameters that must be tuned for better performance:

* `max_grad_norm`: The maximum L2 norm of per-sample gradients before they are aggregated by the averaging step.
* `noise_multiplier`: The amount of noise sampled and added to the average of the gradients in a batch.
* `target_epsilon` and `target_delta`: The target ϵ and δ of the (ϵ,δ)-differential privacy guarantee. 

It is worth noting that in order to use the opacus `PrivacyEngine` class we need to properly define as training plan attributes a `model`, a `dataloader` and an `optimizer`.

In [4]:
%%writefile "$model_file"

import torch
import torch.nn as nn
from fedbiomed.common.torchnn import TorchTrainingPlan
from fedbiomed.common.logger import logger
from torch.utils.data import DataLoader
import torch.optim as optim
from torchvision import datasets, transforms, models
from opacus import PrivacyEngine 
from opacus.validators import ModuleValidator
from opacus.utils.batch_memory_manager import BatchMemoryManager
from typing import Union, List
from tqdm import tqdm

# Here we define the model to be used. 
# You can use any class name (here 'Net')
class CIFAR10DPPlan(TorchTrainingPlan):
    def __init__(self, model_args):
        super(CIFAR10DPPlan, self).__init__()
        
        # Here we define the custom dependencies that will be needed by our custom Dataloader
        # In this case, we need the torch DataLoader classes
        # Since we will train on MNIST, we need datasets and transform from torchvision
        deps = ["from torchvision import datasets, transforms, models",
                "from torch.utils.data import DataLoader",
                "import torch.optim as optim",
                "from fedbiomed.common.logger import logger",
                "from typing import Union, List",
                "from tqdm import tqdm",
                "from opacus import PrivacyEngine",
                "from opacus.validators import ModuleValidator",
                "from opacus.utils.batch_memory_manager import BatchMemoryManager",]
        self.add_dependency(deps)
        
        self.model = models.resnet18(num_classes=model_args['num_classes'])
        self.model = ModuleValidator.fix(self.model)
        ModuleValidator.validate(self.model, strict=False)
        
        self.loss = nn.CrossEntropyLoss()
        
        self.max_grad_norm = model_args['max_grad_norm']
        self.epsilon = model_args['target_epsilon']
        self.delta = model_args['target_delta']
        self.max_physical_batch_size = model_args['max_physical_batch_size']

    def forward(self, x):
        return self.model(x)

    def training_data(self, batch_size = 48):
        CIFAR10_MEAN = (0.4914, 0.4822, 0.4465)
        CIFAR10_STD_DEV = (0.2023, 0.1994, 0.2010)
        # Custom torch Dataloader for CIFAR data
        transform = transforms.Compose([transforms.ToTensor(),
                                        transforms.Normalize(CIFAR10_MEAN, CIFAR10_STD_DEV),
                                       ])
        dataset1 = datasets.CIFAR10(self.dataset_path, train=True, download=False, transform=transform)
        train_kwargs = {'batch_size': batch_size, 'shuffle': True}
        data_loader = torch.utils.data.DataLoader(dataset1, **train_kwargs)
        return data_loader
    
    def training_step(self, data, target):
        output = self.forward(data)
        loss   = self.loss(output, target)
        return loss
    
    def training_routine(self,
                         epochs: int = 2,
                         log_interval: int = 10,
                         lr: Union[int, float] = 1e-3,
                         batch_size: int = 48,
                         batch_maxnum: int = 0,
                         dry_run: bool = False,
                         monitor=None):
        
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        
        self.optimizer = optim.RMSprop(self.model.parameters(), lr=lr)
        
        training_data = self.training_data(batch_size=batch_size)
        
        # enter PrivacyEngine
        privacy_engine = PrivacyEngine()
        self.model, self.optimizer, training_data = privacy_engine.make_private_with_epsilon(
                                                                    module=self.model,
                                                                    optimizer=self.optimizer,
                                                                    data_loader=training_data,
                                                                    epochs=epochs,
                                                                    target_epsilon=self.epsilon,
                                                                    target_delta=self.delta,
                                                                    max_grad_norm=self.max_grad_norm,
                                                                )

        for epoch in range(1, epochs + 1):
            self.model.train()
            # (below) sampling data (with `training_data` method defined on
            # researcher's notebook)
            #for batch_idx, (data, target) in enumerate(tqdm(training_data)):
            
            # 
            with BatchMemoryManager(data_loader=training_data, 
                                    max_physical_batch_size=self.max_physical_batch_size, 
                                    optimizer=self.optimizer
                                    ) as memory_safe_data_loader:

                for batch_idx, (data, target) in enumerate(memory_safe_data_loader):
            
                    #self.model.train()  # model training
                    data, target = data.to(self.device), target.to(self.device)
                    self.optimizer.zero_grad()
                    # (below) calling method `training_step` defined on
                    # researcher's notebook
                    res = self.training_step(data, target)
                    res.backward()
                    self.optimizer.step()

                    # do not take into account more than batch_maxnum
                    # batches from the dataset
                    if (batch_maxnum > 0) and (batch_idx >= batch_maxnum):
                        #print('Reached {} batches for this epoch, ignore remaining data'.format(batch_maxnum))
                        logger.debug('Reached {} batches for this epoch, ignore remaining data'.format(batch_maxnum))
                        break

                    if batch_idx % log_interval == 0:
                        logger.info('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                            epoch,
                            batch_idx * len(data),
                            len(training_data.dataset),
                            100 * batch_idx / len(training_data),
                            res.item()))
                        eps = privacy_engine.get_epsilon(self.delta)
                        logger.info('Epsilon={:.2f}, Delta={}'.format(eps,self.delta))

                        # Send scalar values via general/feedback topic
                        if monitor is not None:
                            monitor.add_scalar('Loss', res.item(), batch_idx, epoch)

                        if dry_run:
                            return

    def save(self, filename, params: dict = None) -> None:
        if params is not None:

            # params keys are changed by the privacy engine (as _module.param_key): should be re-named
            params_keys = list(params.keys())
            for key in params_keys:
                if '_module' in key:
                    newkey = key.replace('_module.', '')
                    params[newkey] = params.pop(key)
                    
            return(torch.save(params, filename))
        else:
            return torch.save(self.state_dict(), filename)

Writing /Users/balelli/ownCloud/INRIA_EPIONE/FedBioMed/fedbiomed/var/tmp/tmp2jx06stj/Cifar_opacus.py


This group of arguments correspond respectively:
* `model_args`: a dictionary with the arguments related to the model (e.g. number of layers, features, etc.). This will be passed to the model class on the node side. For instance, the privacy parameters should be passed here.
* `training_args`: a dictionary containing the arguments for the training routine (e.g. batch size, learning rate, epochs, etc.). This will be passed to the routine on the node side.

**NOTE:** typos and/or lack of positional (required) arguments will raise error. 🤓

In [5]:
model_args = {'num_classes': 10, 'max_grad_norm': 1.2, 
              'target_epsilon': 50.0, 'target_delta': 1e-5,
              'max_physical_batch_size': 100}

training_args = {
    'batch_size': 50, 
    'lr': 1e-3, 
    'epochs': 1, 
    'dry_run': False,  
    'batch_maxnum': 100 # Fast pass for development : only use ( batch_maxnum * batch_size ) samples
}

# Train the federated model

Define an experiment
- search nodes serving data for these `tags`, optionally filter on a list of node ID with `nodes`
- run a round of local training on nodes with model defined in `model_path` + federation with `aggregator`
- run for `rounds` rounds, applying the `node_selection_strategy` between the rounds

In [6]:
from fedbiomed.researcher.experiment import Experiment
from fedbiomed.researcher.aggregators.fedavg import FedAverage

tags =  ['#CIFAR10', '#dataset']
rounds = 2

exp = Experiment(tags=tags,
                 #nodes=None,
                 model_path=model_file,
                 model_args=model_args,
                 model_class='CIFAR10DPPlan',
                 training_args=training_args,
                 rounds=rounds,
                 aggregator=FedAverage(),
                 node_selection_strategy=None)

2022-01-19 18:46:17,430 fedbiomed INFO - Searching dataset with data tags: ['#CIFAR10', '#dataset'] for all nodes
2022-01-19 18:46:17,441 fedbiomed INFO - log from: node_558e6d2e-b8c0-4f8f-bf9a-ad27cc58f787 / DEBUG - Message received: {'researcher_id': 'researcher_24bd8fc8-05ae-4d8a-914e-7eb6b2591345', 'tags': ['#CIFAR10', '#dataset'], 'command': 'search'}
2022-01-19 18:46:17,442 fedbiomed INFO - log from: node_8c851678-c66f-41b1-a309-80263ab6cab1 / DEBUG - Message received: {'researcher_id': 'researcher_24bd8fc8-05ae-4d8a-914e-7eb6b2591345', 'tags': ['#CIFAR10', '#dataset'], 'command': 'search'}
2022-01-19 18:46:27,452 fedbiomed INFO - Node selected for training -> node_558e6d2e-b8c0-4f8f-bf9a-ad27cc58f787
2022-01-19 18:46:27,453 fedbiomed INFO - Node selected for training -> node_8c851678-c66f-41b1-a309-80263ab6cab1
2022-01-19 18:46:27,456 fedbiomed INFO - Checking data quality of federated datasets...
2022-01-19 18:46:28,501 fedbiomed DEBUG - torchnn saved model filename: /Users/bal

Let's start the experiment.

By default, this function doesn't stop until all the `rounds` are done for all the nodes

In [7]:
exp.run()

2022-01-19 18:46:42,832 fedbiomed INFO - Sampled nodes in round 0 ['node_558e6d2e-b8c0-4f8f-bf9a-ad27cc58f787', 'node_8c851678-c66f-41b1-a309-80263ab6cab1']
01/19/2022 18:46:42:INFO:Sampled nodes in round 0 ['node_558e6d2e-b8c0-4f8f-bf9a-ad27cc58f787', 'node_8c851678-c66f-41b1-a309-80263ab6cab1']
2022-01-19 18:46:42,834 fedbiomed INFO - Send message to node node_558e6d2e-b8c0-4f8f-bf9a-ad27cc58f787 - {'researcher_id': 'researcher_24bd8fc8-05ae-4d8a-914e-7eb6b2591345', 'job_id': '012184b0-7525-4aff-b000-037246f1a1d2', 'training_args': {'batch_size': 50, 'lr': 0.001, 'epochs': 1, 'dry_run': False, 'batch_maxnum': 100}, 'model_args': {'num_classes': 10, 'max_grad_norm': 1.2, 'target_epsilon': 50.0, 'target_delta': 1e-05, 'max_physical_batch_size': 100}, 'command': 'train', 'model_url': 'http://localhost:8844/media/uploads/2022/01/19/my_model_d4cc67e5-d4a5-4174-b28a-5e1850a51a1e.py', 'params_url': 'http://localhost:8844/media/uploads/2022/01/19/aggregated_params_init_180d65d5-ecfc-473c-bcd

01/19/2022 18:46:42:INFO:log from: node_8c851678-c66f-41b1-a309-80263ab6cab1 / DEBUG - Message received: {'researcher_id': 'researcher_24bd8fc8-05ae-4d8a-914e-7eb6b2591345', 'job_id': '012184b0-7525-4aff-b000-037246f1a1d2', 'training_args': {'batch_size': 50, 'lr': 0.001, 'epochs': 1, 'dry_run': False, 'batch_maxnum': 100}, 'model_args': {'num_classes': 10, 'max_grad_norm': 1.2, 'target_epsilon': 50.0, 'target_delta': 1e-05, 'max_physical_batch_size': 100}, 'command': 'train', 'model_url': 'http://localhost:8844/media/uploads/2022/01/19/my_model_d4cc67e5-d4a5-4174-b28a-5e1850a51a1e.py', 'params_url': 'http://localhost:8844/media/uploads/2022/01/19/aggregated_params_init_180d65d5-ecfc-473c-bcde-25795617a769.pt', 'model_class': 'CIFAR10DPPlan', 'training_data': {'node_8c851678-c66f-41b1-a309-80263ab6cab1': ['dataset_2e3575d8-40a3-421a-9194-a217e0bc92c1']}}
2022-01-19 18:46:42,919 fedbiomed INFO - log from: node_8c851678-c66f-41b1-a309-80263ab6cab1 / DEBUG - [TASKS QUEUE] Item:{'researche

01/19/2022 18:50:35:INFO:log from: node_8c851678-c66f-41b1-a309-80263ab6cab1 / INFO - Epsilon=22.21, Delta=1e-05
2022-01-19 18:51:33,491 fedbiomed INFO - log from: node_8c851678-c66f-41b1-a309-80263ab6cab1 / INFO - Train Epoch: 1 [2120/50000 (4%)]	Loss: 2.180702
01/19/2022 18:51:33:INFO:log from: node_8c851678-c66f-41b1-a309-80263ab6cab1 / INFO - Train Epoch: 1 [2120/50000 (4%)]	Loss: 2.180702
2022-01-19 18:51:33,643 fedbiomed INFO - log from: node_8c851678-c66f-41b1-a309-80263ab6cab1 / INFO - Epsilon=23.38, Delta=1e-05
01/19/2022 18:51:33:INFO:log from: node_8c851678-c66f-41b1-a309-80263ab6cab1 / INFO - Epsilon=23.38, Delta=1e-05
2022-01-19 18:51:40,177 fedbiomed INFO - log from: node_558e6d2e-b8c0-4f8f-bf9a-ad27cc58f787 / INFO - Train Epoch: 1 [2080/50000 (4%)]	Loss: 2.218732
01/19/2022 18:51:40:INFO:log from: node_558e6d2e-b8c0-4f8f-bf9a-ad27cc58f787 / INFO - Train Epoch: 1 [2080/50000 (4%)]	Loss: 2.218732
2022-01-19 18:51:40,352 fedbiomed INFO - log from: node_558e6d2e-b8c0-4f8f-bf

01/19/2022 19:06:44:INFO:Downloading model params after training on node_8c851678-c66f-41b1-a309-80263ab6cab1 - from http://localhost:8844/media/uploads/2022/01/19/node_params_780f1f5d-341c-476e-b28e-d2b66ebddd81.pt
2022-01-19 19:06:46,350 fedbiomed INFO - Nodes that successfully reply in round 0 ['node_558e6d2e-b8c0-4f8f-bf9a-ad27cc58f787', 'node_8c851678-c66f-41b1-a309-80263ab6cab1']
01/19/2022 19:06:46:INFO:Nodes that successfully reply in round 0 ['node_558e6d2e-b8c0-4f8f-bf9a-ad27cc58f787', 'node_8c851678-c66f-41b1-a309-80263ab6cab1']
2022-01-19 19:06:53,199 fedbiomed INFO - Saved aggregated params for round 0 in /Users/balelli/ownCloud/INRIA_EPIONE/FedBioMed/fedbiomed/var/experiments/Experiment_0000/aggregated_params_815fc09e-ca28-4dd5-93b7-743d596774ec.pt
01/19/2022 19:06:53:INFO:Saved aggregated params for round 0 in /Users/balelli/ownCloud/INRIA_EPIONE/FedBioMed/fedbiomed/var/experiments/Experiment_0000/aggregated_params_815fc09e-ca28-4dd5-93b7-743d596774ec.pt
2022-01-19 19:06

2022-01-19 19:06:53,313 fedbiomed INFO - log from: node_558e6d2e-b8c0-4f8f-bf9a-ad27cc58f787 / DEBUG - Message received: {'researcher_id': 'researcher_24bd8fc8-05ae-4d8a-914e-7eb6b2591345', 'job_id': '012184b0-7525-4aff-b000-037246f1a1d2', 'training_args': {'batch_size': 50, 'lr': 0.001, 'epochs': 1, 'dry_run': False, 'batch_maxnum': 100}, 'model_args': {'num_classes': 10, 'max_grad_norm': 1.2, 'target_epsilon': 50.0, 'target_delta': 1e-05, 'max_physical_batch_size': 100}, 'command': 'train', 'model_url': 'http://localhost:8844/media/uploads/2022/01/19/my_model_d4cc67e5-d4a5-4174-b28a-5e1850a51a1e.py', 'params_url': 'http://localhost:8844/media/uploads/2022/01/19/aggregated_params_815fc09e-ca28-4dd5-93b7-743d596774ec.pt', 'model_class': 'CIFAR10DPPlan', 'training_data': {'node_558e6d2e-b8c0-4f8f-bf9a-ad27cc58f787': ['dataset_ebb64353-c02f-4f63-9862-578c29bb89db']}}
01/19/2022 19:06:53:INFO:log from: node_558e6d2e-b8c0-4f8f-bf9a-ad27cc58f787 / DEBUG - Message received: {'researcher_id':

01/19/2022 19:12:23:INFO:log from: node_8c851678-c66f-41b1-a309-80263ab6cab1 / INFO - Train Epoch: 1 [1620/50000 (3%)]	Loss: 2.188330
2022-01-19 19:12:24,106 fedbiomed INFO - log from: node_8c851678-c66f-41b1-a309-80263ab6cab1 / INFO - Epsilon=22.21, Delta=1e-05
01/19/2022 19:12:24:INFO:log from: node_8c851678-c66f-41b1-a309-80263ab6cab1 / INFO - Epsilon=22.21, Delta=1e-05
2022-01-19 19:12:27,139 fedbiomed INFO - log from: node_558e6d2e-b8c0-4f8f-bf9a-ad27cc58f787 / INFO - Train Epoch: 1 [1620/50000 (3%)]	Loss: 2.343717
01/19/2022 19:12:27:INFO:log from: node_558e6d2e-b8c0-4f8f-bf9a-ad27cc58f787 / INFO - Train Epoch: 1 [1620/50000 (3%)]	Loss: 2.343717
2022-01-19 19:12:27,285 fedbiomed INFO - log from: node_558e6d2e-b8c0-4f8f-bf9a-ad27cc58f787 / INFO - Epsilon=22.21, Delta=1e-05
01/19/2022 19:12:27:INFO:log from: node_558e6d2e-b8c0-4f8f-bf9a-ad27cc58f787 / INFO - Epsilon=22.21, Delta=1e-05
2022-01-19 19:13:16,616 fedbiomed INFO - log from: node_8c851678-c66f-41b1-a309-80263ab6cab1 / INF

01/19/2022 19:19:07:INFO:Downloading model params after training on node_558e6d2e-b8c0-4f8f-bf9a-ad27cc58f787 - from http://localhost:8844/media/uploads/2022/01/19/node_params_1483a8a7-6461-4cdb-9f29-dda639d2f1df.pt
2022-01-19 19:19:10,079 fedbiomed INFO - log from: node_8c851678-c66f-41b1-a309-80263ab6cab1 / INFO - results uploaded successfully 
01/19/2022 19:19:10:INFO:log from: node_8c851678-c66f-41b1-a309-80263ab6cab1 / INFO - results uploaded successfully 
2022-01-19 19:19:21,873 fedbiomed INFO - Downloading model params after training on node_8c851678-c66f-41b1-a309-80263ab6cab1 - from http://localhost:8844/media/uploads/2022/01/19/node_params_2f0486f0-3312-468c-86d0-503f8cad45d0.pt
01/19/2022 19:19:21:INFO:Downloading model params after training on node_8c851678-c66f-41b1-a309-80263ab6cab1 - from http://localhost:8844/media/uploads/2022/01/19/node_params_2f0486f0-3312-468c-86d0-503f8cad45d0.pt
2022-01-19 19:19:24,136 fedbiomed INFO - Nodes that successfully reply in round 1 ['no

Different timings (in seconds) are reported for each dataset of a node participating in a round :
- `rtime_training` real time (clock time) spent in the training function on the node
- `ptime_training` process time (user and system CPU) spent in the training function on the node
- `rtime_total` real time (clock time) spent in the researcher between sending the request and handling the response, at the `Job()` layer

In [8]:
print("\nList the training rounds : ", exp.training_replies.keys())

print("\nList the nodes for the last training round and their timings : ")
round_data = exp.training_replies[rounds - 1].data
for c in range(len(round_data)):
    print("\t- {id} :\
    \n\t\trtime_training={rtraining:.2f} seconds\
    \n\t\tptime_training={ptraining:.2f} seconds\
    \n\t\trtime_total={rtotal:.2f} seconds".format(id = round_data[c]['node_id'],
        rtraining = round_data[c]['timing']['rtime_training'],
        ptraining = round_data[c]['timing']['ptime_training'],
        rtotal = round_data[c]['timing']['rtime_total']))
print('\n')


List the training rounds :  dict_keys([0, 1])

List the nodes for the last training round and their timings : 
	- node_558e6d2e-b8c0-4f8f-bf9a-ad27cc58f787 :    
		rtime_training=626.72 seconds    
		ptime_training=769.41 seconds    
		rtime_total=650.74 seconds
	- node_8c851678-c66f-41b1-a309-80263ab6cab1 :    
		rtime_training=634.77 seconds    
		ptime_training=785.68 seconds    
		rtime_total=665.53 seconds




# Test Model

We define a little testing routine to extract the accuracy metrics on the testing dataset
## Important
This is done to test the model because it can be accessed in a developpement environment  
In production, the data wont be accessible on the nodes, need a test dataset on the server or accessible from the server.

In [9]:

import torch
import torch.nn as nn

import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
from PIL import Image
import os

def testing_Accuracy(model, data_loader):
    model.eval()
    test_loss = 0
    correct = 0
    
    device = "cpu"

    correct = 0
    
    loader_size = len(data_loader)
    with torch.no_grad():
        for idx, (data, target) in enumerate(data_loader):
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()
            
            #only uses 10% of the dataset, results are similar but faster
            if idx >= loader_size / 10:
                pass
                break

    
        pred = output.argmax(dim=1, keepdim=True)

    test_loss /= len(data_loader.dataset)
    accuracy = 100* correct/(data_loader.batch_size * idx)

    return(test_loss, accuracy)

Test dataset

In [10]:
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import CIFAR10
import os

# These values, specific to the CIFAR10 dataset, are assumed to be known.
# If necessary, they can be computed with modest privacy budget.
CIFAR10_MEAN = (0.4914, 0.4822, 0.4465)
CIFAR10_STD_DEV = (0.2023, 0.1994, 0.2010)

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(CIFAR10_MEAN, CIFAR10_STD_DEV),
])

base_dir = tmp_dir_model.name 
if not os.path.isdir(os.path.join(base_dir, "cifar10")):
    os.makedirs(os.path.join(base_dir, "cifar10"))
test_data_dir = os.path.join(base_dir, "cifar10")

test_dataset = CIFAR10(
    root=test_data_dir, train=False, download=True, transform=transform)

test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=48,
    shuffle=False,
)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting /Users/balelli/ownCloud/INRIA_EPIONE/FedBioMed/fedbiomed/var/tmp/tmp2jx06stj/cifar10/cifar-10-python.tar.gz to /Users/balelli/ownCloud/INRIA_EPIONE/FedBioMed/fedbiomed/var/tmp/tmp2jx06stj/cifar10


We define a util function to calculate the accuracy:

In [11]:
def accuracy(preds, labels):
    return (preds == labels).mean()

We define the model, and we assign to it the model parameters estimated at the last federated optimization round.

In [12]:
from torchvision import models
from opacus.validators import ModuleValidator

model = models.resnet18(num_classes=10)
model = ModuleValidator.fix(model)
ModuleValidator.validate(model, strict=False)

model = exp.model_instance
model.load_state_dict(exp.aggregated_params[rounds - 1]['params'])

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

We define a function to validate our model on our test dataset.

In [13]:
def test(model, test_loader, device):
    model.eval()
    criterion = nn.CrossEntropyLoss()
    losses = []
    top1_acc = []

    with torch.no_grad():
        for images, target in test_loader:
            images = images.to(device)
            target = target.to(device)

            output = model(images)
            loss = criterion(output, target)
            preds = np.argmax(output.detach().cpu().numpy(), axis=1)
            labels = target.detach().cpu().numpy()
            acc = accuracy(preds, labels)

            losses.append(loss.item())
            top1_acc.append(acc)

    top1_avg = np.mean(top1_acc)

    print(
        f"\tTest set:"
        f"Loss: {np.mean(losses):.6f} "
        f"Acc: {top1_avg * 100:.6f} "
    )
    return np.mean(top1_acc)

And we finally test our model!

In [14]:
top1_acc = test(model, test_loader, device)

	Test set:Loss: 2.189102 Acc: 17.663477 
